In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os

## Read data in

In [61]:
trades_1 = pd.read_csv("round5/trades_round_5_day_2.csv", delimiter=';')
trades_2 = pd.read_csv("round5/trades_round_5_day_3.csv", delimiter=';')
trades_2['timestamp'] += 1e6
trades_3 = pd.read_csv("round5/trades_round_5_day_4.csv", delimiter=';')
trades_3['timestamp'] += 2e6

prices_1 = pd.read_csv("round5/prices_round_5_day_2.csv", delimiter=';')
prices_2 = pd.read_csv("round5/prices_round_5_day_3.csv", delimiter=';')
prices_2['timestamp'] += 1e6
prices_3 = pd.read_csv("round5/prices_round_5_day_4.csv", delimiter=';')
prices_3['timestamp'] += 2e6

In [62]:
prices = pd.concat([prices_1, prices_2, prices_3])
trades = pd.concat([trades_1, trades_2, trades_3])

def simple_mid_price(row):
    bid = row[3]
    ask = row[9]
    if np.isnan(bid) and np.isnan(ask):
        return 0
    elif np.isnan(bid):
        return ask
    elif np.isnan(ask):
        return bid
    return (bid+ask) / 2

prices['mid_price'] = prices.apply(simple_mid_price, axis=1)

C:\Users\nirav\AppData\Local\Temp\ipykernel_69660\502764631.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bid = row[3]
C:\Users\nirav\AppData\Local\Temp\ipykernel_69660\502764631.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ask = row[9]


In [63]:
buyers = set(trades['buyer'].unique())
sellers = set(trades['seller'].unique())

print("buyers: ", len(buyers))
print(buyers)
print("sellers: ", len(sellers))
print(sellers)

buyers_and_sellers = set.union(buyers, sellers)
print("buyers and sellers: ", len(buyers_and_sellers))

print(buyers_and_sellers)


buyers:  10
{'Pablo', 'Paris', 'Penelope', 'Peter', 'Olivia', 'Gary', 'Charlie', 'Caesar', 'Camilla', 'Gina'}
sellers:  11
{'Pablo', 'Paris', 'Penelope', 'Peter', 'Olivia', 'Gary', 'Olga', 'Charlie', 'Caesar', 'Camilla', 'Gina'}
buyers and sellers:  11
{'Pablo', 'Paris', 'Olivia', 'Caesar', 'Olga', 'Penelope', 'Peter', 'Gary', 'Charlie', 'Camilla', 'Gina'}


## Backtesting function

In [91]:
from tqdm.notebook import tqdm

def backtest_trader(trader, product, trades, prices):
    trader_df = trades[(trades['buyer'] == trader) | (trades['seller'] == trader)]
    trader_df = trader_df[trader_df['symbol'] == product]
    product_prices = prices[prices['product'] == product]

    position = 0
    # start the DCA at the very first mid_price instead of 0
    if not product_prices.empty:
        avg_entry_price = product_prices.iloc[0]['mid_price']
    else:
        avg_entry_price = 0.0
    buy_costs = 0
    sell_costs = 0
    pnls = []
    dcas = []
    timestamps = []
    positions = []

    
    prev_pnl = 0.0

    for row in tqdm(product_prices.itertuples(index=False),
                    total=len(product_prices),
                    desc=f"        {product}",
                    leave=False, position=2):
        time = row.timestamp
        mid_price = row.mid_price

        trades_at_time = trader_df[trader_df['timestamp'] == time]

        for _, trade in trades_at_time.iterrows():
            qty = trade['quantity']
            price = trade['price']

            if trade['buyer'] == trader:
                # Buying
                if position >= 0:
                    # Increase long position
                    total_cost = avg_entry_price * position + price * qty
                    position += qty
                    avg_entry_price = total_cost / position
                else:
                    # Closing short or flipping to long
                    if qty > abs(position):
                        # Flip to long
                        leftover_qty = qty + position
                        avg_entry_price = price  # new long starts fresh
                        position = leftover_qty
                    else:
                        position += qty  # reduce short
                        if position == 0:
                            avg_entry_price = mid_price  # flatten

            elif trade['seller'] == trader:
                # Selling
                if position <= 0:
                    # Increase short position
                    total_cost = avg_entry_price * abs(position) + price * qty
                    position -= qty
                    avg_entry_price = total_cost / abs(position)
                else:
                    # Closing long or flipping to short
                    if qty > position:
                        # Flip to short
                        leftover_qty = qty - position
                        avg_entry_price = price  # new short starts fresh
                        position = -leftover_qty
                    else:
                        position -= qty  # reduce long
                        if position == 0:
                            avg_entry_price = mid_price  # flatten

        # Calculate PnL
        if position > 0:
            pnl = (mid_price - avg_entry_price) * position
        elif position < 0:
            pnl = (avg_entry_price - mid_price) * abs(position)
        else:
            pnl = 0

        pnl += prev_pnl

        pnls.append(pnl)
        dcas.append(avg_entry_price)
        timestamps.append(time)
        positions.append(position)

        if time % 1e6 == 0:
            prev_pnl = pnl
            position = 0
            avg_entry_price = mid_price

    return pnls, timestamps, positions, dcas



## Plotting function

In [102]:
import numpy as np
import matplotlib.pyplot as plt

def plot_trader_dca_signals(trader, product, trades, prices):
    # backtest gives us PnL, timestamps, positions, dcas
    pnls, timestamps, positions, dcas = backtest_trader(trader, product, trades, prices)

    # extract buys/sells for arrow overlays
    trader_df = trades[(trades['buyer']==trader) | (trades['seller']==trader)]
    tdf = trader_df[trader_df['symbol']==product]
    buy_df  = tdf[tdf['buyer']==trader]
    sell_df = tdf[tdf['seller']==trader]

    price_df = prices[prices['product']==product]

    fig, (ax0, ax1) = plt.subplots(2,1,sharex=True, figsize=(14,10), gridspec_kw={'height_ratios':[2,1]})

    # --- panel 0: DCA vs Mid Price + long/short shading + signals ---
    # shading
    ts = np.array(timestamps)
    pos = np.array(positions)
    ymin, ymax = price_df['mid_price'].min()*0.999, price_df['mid_price'].max()*1.001

    ax0.fill_between(ts, ymin, ymax, where=pos>0, facecolor='green',  alpha=0.1, label='Long Region')
    ax0.fill_between(ts, ymin, ymax, where=pos<0, facecolor='red',    alpha=0.1, label='Short Region')

    # mid price
    ax0.plot(price_df['timestamp'], price_df['mid_price'],
             c='purple', linewidth=1, alpha=0.7, label='Mid Price')

    # DCA
    ax0.plot(timestamps, dcas,
             c='orange', linewidth=2, label='Dollar Cost Avg')

    # buy/sell arrows
    # up-arrows for buys
    ax0.scatter(
        buy_df['timestamp'], buy_df['price'],
        marker='^', s=30, c='green', alpha=0.6, label='Buy Signal'
    )
    # down-arrows for sells
    ax0.scatter(
        sell_df['timestamp'], sell_df['price'],
        marker='v', s=30, c='red', alpha=0.6, label='Sell Signal'
    )

    ax0.set_title(f"{trader} DCA vs Mid Price & Signals for {product}")
    ax0.set_ylabel("Price")
    ax0.legend(loc='upper left')
    ax0.grid(True)

    # --- panel 1: PnL & Position Size ---
    ax1.plot(timestamps, pnls,
             c='blue', linewidth=1, label='PnL')
    ax1.set_ylabel("PnL", color='blue')
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.plot(timestamps, positions,
             c='red', linewidth=1, alpha=0.7, label='Position Size')
    ax2.set_ylabel("Position Size", color='red')

    ax1.set_title(f"{trader} PnL & Position for {product}")
    ax1.set_xlabel("Timestamp")

    # merge legends
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines+lines2, labels+labels2, loc='upper left')

    plt.tight_layout()
    # plt.show()
    os.makedirs(f"product_plots/{product}", exist_ok=True)
    plt.savefig(f"product_plots/{product}/{trader}.png")
    plt.close()

In [ ]:
for trader in tqdm(buyers_and_sellers, desc=f"Generating", position=0):
    # when buyer or seller is the trader
    sell_trades_df = trades[(trades['buyer'] == trader)]
    buy_trades_df = trades[(trades['seller'] == trader)]

    products = set.union(set(sell_trades_df['symbol'].unique()), set(buy_trades_df['symbol'].unique()))

    for product in tqdm(products, desc=f"    {trader}", leave=False, position=1):
        plot_trader_dca_signals(trader, product, trades, prices)

Generating:   0%|          | 0/11 [00:00<?, ?it/s]

    Pablo:   0%|          | 0/11 [00:00<?, ?it/s]

        PICNIC_BASKET2:   0%|          | 0/30000 [00:00<?, ?it/s]

        VOLCANIC_ROCK_VOUCHER_10000:   0%|          | 0/30000 [00:00<?, ?it/s]

        VOLCANIC_ROCK_VOUCHER_10500:   0%|          | 0/30000 [00:00<?, ?it/s]

...

        JAMS:   0%|          | 0/30000 [00:00<?, ?it/s]

    Gina:   0%|          | 0/3 [00:00<?, ?it/s]

        SQUID_INK:   0%|          | 0/30000 [00:00<?, ?it/s]

        RAINFOREST_RESIN:   0%|          | 0/30000 [00:00<?, ?it/s]

        KELP:   0%|          | 0/30000 [00:00<?, ?it/s]

## Grid search

In [122]:
def compute_trade_impact(
    mid_price_series: pd.Series,
    trades_df: pd.DataFrame,
    lookahead_lags: list[int] = [100,200,400,800,1600,3200,6400]
) -> pd.Series:
    """
    Compute size‑weighted signed trade‐impact correlations at multiple look‑ahead lags.
    
    mid_price_series : pd.Series
        indexed by timestamp, gives the mid‐price at each time.
    trades_df : pd.DataFrame
        must contain columns ['timestamp','quantity','buyer','seller'].
    lookahead_lags : list of ints
        how many seconds (or same units as your timestamps) to look ahead.
        
    Returns
    -------
    pd.Series indexed by lag, value = Pearson r between signed_size and forward returns.
    """
    # extract price data as numpy for speed
    price_times  = mid_price_series.index.values
    price_values = mid_price_series.values

    # prepare results container
    impacts = {}

    # precompute arrays of event data
    ev_times  = trades_df['timestamp'].to_numpy()
    ev_sizes  = trades_df['quantity'].to_numpy()
    # +1 for buys, –1 for sells
    ev_sides  = np.where(trades_df['buyer'] == trades_df['buyer'],  # always True
                         np.where(trades_df['buyer'] == trades_df['buyer'],  # dummy to get shape
                                  1, -1), -1)  
    # Actually do:
    ev_signs = np.where(trades_df['buyer'] == trades_df['buyer'], 1, -1)
    # Above we just need to detect if this row is a buy or a sell for our trader;
    # Assuming you filter for one trader, use:
    # ev_signs = np.where(trades_df['buyer']==trader, +1, -1)

    # If you want to weight by size and side:
    signed_sizes = ev_sizes * ev_signs

    # for each look‑ahead horizon
    for lag in lookahead_lags:
        sigs = []
        rets = []

        for t0, ss in zip(ev_times, signed_sizes):
            # find the price‐bar index >= trade time
            i = np.searchsorted(price_times, t0)
            if i >= len(price_times):
                continue

            t_target = price_times[i] + lag
            j = np.searchsorted(price_times, t_target)
            if j < len(price_times):
                p0 = price_values[i]
                p1 = price_values[j]
                # safety against zero‐division
                if p0 <= 0:
                    continue
                forward_ret = (p1 - p0) / p0
                sigs.append(ss)
                rets.append(forward_ret)

        if len(sigs) >= 2:
            impacts[lag] = pd.Series(sigs).corr(pd.Series(rets))
        else:
            impacts[lag] = np.nan

    return pd.Series(impacts).rename("impact_r")

In [140]:
from tqdm.notebook import tqdm

results = []
for product in tqdm(prices['product'].unique(), desc="Product"):
    # build the mid‐price series for this product
    mid_series = prices[prices['product']==product] \
                   .set_index('timestamp')['mid_price']
    # all trades in this product
    sub_trades = trades[trades['symbol']==product]

    # group by (buyer, seller)
    for (buyer, seller), grp in tqdm(sub_trades.groupby(['buyer','seller']),
                                     desc=f"  {product}", leave=False):
        # compute impact at multiple horizons
        impact_series = compute_trade_impact(
            mid_series,
            grp,
            lookahead_lags=[100, 200, 400, 800, 1600, 3200, 6400]
        )

        # pack into one row, including product
        row = {
            'product': product,
            'buyer':   buyer,
            'seller':  seller,
            'n_trades': len(grp),
        }
        # append each lag’s r‑value into the row
        row.update(impact_series.to_dict())

        results.append(row)


# build your final DataFrame
df_impacts = pd.DataFrame(results)

# ensure column order
cols = ['product','buyer','seller','n_trades', 100,200,400,800,1600,3200,6400]
df_impacts = df_impacts[cols]


Product:   0%|          | 0/15 [00:00<?, ?it/s]

  PICNIC_BASKET2:   0%|          | 0/13 [00:00<?, ?it/s]

  VOLCANIC_ROCK_VOUCHER_9750:   0%|          | 0/12 [00:00<?, ?it/s]

  RAINFOREST_RESIN:   0%|          | 0/49 [00:00<?, ?it/s]

c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


...

  VOLCANIC_ROCK_VOUCHER_10000:   0%|          | 0/10 [00:00<?, ?it/s]

  VOLCANIC_ROCK_VOUCHER_10500:   0%|          | 0/10 [00:00<?, ?it/s]

c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


  PICNIC_BASKET1:   0%|          | 0/12 [00:00<?, ?it/s]

c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\nirav\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [144]:
df_impacts

,product,buyer,seller,n_trades,100,200,400,800,1600,3200,6400
0,PICNIC_BASKET2,Caesar,Pablo,146,-0.061810,-0.013592,0.048105,0.063474,0.029650,0.008995,-0.006076
1,PICNIC_BASKET2,Caesar,Penelope,144,-0.067812,-0.059815,-0.037125,0.081037,0.049079,-0.063398,0.058012
2,PICNIC_BASKET2,Camilla,Caesar,105,0.055726,0.057651,0.078131,0.018467,0.036438,0.046502,0.098596
3,PICNIC_BASKET2,Camilla,Pablo,217,-0.153575,-0.145276,-0.050069,-0.003866,0.018086,-0.026420,0.023017
4,PICNIC_BASKET2,Camilla,Penelope,164,0.065326,0.062383,0.054678,-0.012118,0.012414,0.065867,0.062227
5,PICNIC_BASKET2,Charlie,Pablo,157,-0.021849,-0.010573,-0.021515,-0.069059,-0.058637,0.084262,0.029725
6,PICNIC_BASKET2,Charlie,Penelope,81,0.061158,0.019134,0.051253,0.055896,-0.022782,-0.140464,-0.145182
7,PICNIC_BASKET2,Pablo,Caesar,86,0.079939,0.145868,0.042539,0.093206,0.003224,-0.162954,-0.139961
8,PICNIC_BASKET2,Pablo,Camilla,121,-0.119876,-0.089758,-0.053036,-0.024763,-0.104419,-0.149911,-0.079301
9,PICNIC_BASKET2,Pablo,Penelope,5,0.280799,0.027874,0.539452,0.483168,0.526647,0.854009,0.912396


In [145]:
pd.set_option('display.max_rows', 1000)
lags = [100, 200, 400, 800, 1600, 3200, 6400]
df = df_impacts.dropna(subset=lags)

# 2) flag “alpha” whenever any lag’s abs(value) > 0.75
df['alpha'] = df[lags].abs().gt(0.75).any(axis=1)

# keep only the “alpha” rows
df = df[df['alpha']].reset_index(drop=True)

# inspect
df

C:\Users\nirav\AppData\Local\Temp\ipykernel_69660\17051051.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alpha'] = df[lags].abs().gt(0.75).any(axis=1)


,product,buyer,seller,n_trades,100,200,400,800,1600,3200,6400,alpha
0,PICNIC_BASKET2,Pablo,Penelope,5,0.280799,0.027874,0.539452,0.483168,0.526647,0.854009,0.912396,True
1,VOLCANIC_ROCK_VOUCHER_9750,Pablo,Camilla,4,-0.205839,-0.937681,-0.842207,-0.735669,-0.817405,0.716729,0.449642,True
2,SQUID_INK,Charlie,Olivia,3,0.999320,0.999959,0.998249,0.859392,-0.419994,0.535915,0.918122,True
3,SQUID_INK,Gary,Gina,4,-0.523331,-0.368622,-0.560564,-0.955458,-0.433948,-0.707900,0.805539,True
4,SQUID_INK,Gina,Caesar,6,0.861074,0.550411,0.413920,0.286673,-0.220455,-0.467115,-0.699683,True
5,SQUID_INK,Gina,Gary,2,1.000000,1.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,True
6,SQUID_INK,Pablo,Gary,4,-0.817406,0.890079,-0.531282,0.136109,0.230984,0.708986,0.586058,True
7,SQUID_INK,Penelope,Paris,7,-0.879841,-0.263155,-0.462693,-0.825871,-0.965361,0.101753,0.326281,True
8,VOLCANIC_ROCK_VOUCHER_10250,Caesar,Penelope,44,-0.848314,-0.792531,-0.735031,-0.734158,-0.561335,-0.160195,-0.251077,True
9,VOLCANIC_ROCK_VOUCHER_10250,Pablo,Caesar,7,0.637822,0.639891,0.669025,0.667408,0.864065,0.621500,0.850390,True
